In [26]:
import numpy as np
import pandas as pd

coord = (pd.read_csv('airports.dat', index_col=['AIRPORT'],
                     usecols=['AIRPORT', 'LATITUDE', 'LONGITUDE'])
         .groupby(level=0)
         .first()
         .dropna()
         .sample(n=500, random_state=42)
         .sort_index())

coord.head()

,LATITUDE,LONGITUDE
AIRPORT,,
2CA,35.745556,-119.236389
8F3,33.623889,-101.240833
A08,56.598056,-134.242778
A15,70.718611,-154.388333
A27,64.440000,-144.936389


In [27]:
idx = pd.MultiIndex.from_product([coord.index, coord.index], names=['origin', 'dest'])
pairs = pd.concat([coord.add_suffix('_SRC').reindex(idx, level='origin'),
                   coord.add_suffix('_DST').reindex(idx, level='dest')],
                   axis=1)

pairs.head()

LATITUDE_SRC  LONGITUDE_SRC  LATITUDE_DST  LONGITUDE_DST
origin dest                                                          
2CA    2CA      35.745556    -119.236389     35.745556    -119.236389
       8F3      35.745556    -119.236389     33.623889    -101.240833
       A08      35.745556    -119.236389     56.598056    -134.242778
       A15      35.745556    -119.236389     70.718611    -154.388333
       A27      35.745556    -119.236389     64.440000    -144.936389

In [28]:
idx = idx[idx.get_level_values(0) <= idx.get_level_values(1)]

idx

MultiIndex([('2CA', '2CA'),
            ('2CA', '8F3'),
            ('2CA', 'A08'),
            ('2CA', 'A15'),
            ('2CA', 'A27'),
            ('2CA', 'A56'),
            ('2CA', 'A57'),
            ('2CA', 'A66'),
            ('2CA', 'A6K'),
            ('2CA', 'A71'),
            ...
            ('ZMD', 'ZMD'),
            ('ZMD', 'ZNC'),
            ('ZMD', 'ZSL'),
            ('ZMD', 'ZXO'),
            ('ZNC', 'ZNC'),
            ('ZNC', 'ZSL'),
            ('ZNC', 'ZXO'),
            ('ZSL', 'ZSL'),
            ('ZSL', 'ZXO'),
            ('ZXO', 'ZXO')],
           names=['origin', 'dest'], length=125250)

In [29]:
import math

def gcd_py(lat_src, lng_src, lat_dst, lng_dst):
    earth_radius_km = 6373
    degs_to_rads = math.pi/180.0
    precision = 8 #dp
    
    theta_1 = (90-lat_src) * degs_to_rads
    theta_2 = (90-lat_dst) * degs_to_rads
    
    omega_1 = lng_src * degs_to_rads
    omega_2 = lng_dst * degs_to_rads
    
    cos = (math.sin(theta_1) * math.sin(theta_2) * 
           math.cos(omega_1 - omega_2) + 
           math.cos(theta_1) * math.cos(theta_2))
    
    cos = round(cos, precision)
    arc = math.acos(cos)
    
    return arc * earth_radius_km

In [30]:
%%time
r = pairs.apply(lambda x: gcd_py(x['LATITUDE_SRC'], x['LONGITUDE_SRC'], 
                                 x['LATITUDE_DST'], x['LATITUDE_DST']), axis=1)

Wall time: 5.37 s


In [31]:
r.head()

origin  dest
2CA     2CA     11658.209986
        8F3     11805.567309
        A08      9742.476890
        A15      8152.546542
        A27      8873.080964
dtype: float64

In [17]:
def gcd_vec(lat_src, lng_src, lat_dst, lng_dst):
    earth_radius_km = 6373
    
    theta_1 = np.deg2rad(90 - lat_src)
    theta_2 = np.deg2rad(90 - lat_dst)
    
    omega_1 = np.deg2rad(lng_src)
    omega_2 = np.deg2rad(lng_dst)
    
    cos = (np.sin(theta_1) * np.sin(theta_2) * np.cos(omega_1 - omega_2) +
          np.cos(theta_1) * np.cos(theta_2))
    
    arc = np.arccos(cos)
    
    return arc * earth_radius_km

In [19]:
%%time
r = pairs.apply(lambda x: gcd_py(x['LATITUDE_SRC'], x['LONGITUDE_SRC'], 
                                 x['LATITUDE_DST'], x['LATITUDE_DST']), axis=1)

Wall time: 6.92 s


In [20]:
r.head()

origin  dest
2CA     2CA     11658.209986
        8F3     11805.567309
        A08      9742.476890
        A15      8152.546542
        A27      8873.080964
dtype: float64

In [21]:
%%time
pd.Series([gcd_py(*x) for x in pairs.itertuples(index=False)], index=pairs.index)

Wall time: 1.04 s


origin  dest
2CA     2CA        0.000000
        8F3     1660.364858
        A08     2578.473725
        A15     4404.379276
        A27     3624.913383
                   ...     
ZXO     ZLO     4763.710791
        ZMD     9345.085453
        ZNC     1622.875654
        ZSL     6754.874826
        ZXO        0.000000
Length: 250000, dtype: float64

In [22]:
%%time
pd.Series([gcd_vec(*x) for x in pairs.itertuples(index=False)], index=pairs.index)

<ipython-input-17-33ed6328afff>:13: RuntimeWarning: invalid value encountered in arccos
  arc = np.arccos(cos)


Wall time: 4.78 s


origin  dest
2CA     2CA        0.000095
        8F3     1660.364741
        A08     2578.473679
        A15     4404.379250
        A27     3624.913415
                   ...     
ZXO     ZLO     4763.710784
        ZMD     9345.085432
        ZNC     1622.875569
        ZSL     6754.874795
        ZXO        0.000134
Length: 250000, dtype: float64

In [23]:
%%time
r = gcd_vec(pairs['LATITUDE_SRC'], pairs['LONGITUDE_SRC'],
            pairs['LATITUDE_DST'], pairs['LONGITUDE_DST'])

Wall time: 87 ms


c:\users\turntabl\appdata\local\programs\python\python38\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [24]:
r

origin  dest
2CA     2CA        0.000095
        8F3     1660.364741
        A08     2578.473679
        A15     4404.379250
        A27     3624.913415
                   ...     
ZXO     ZLO     4763.710784
        ZMD     9345.085432
        ZNC     1622.875569
        ZSL     6754.874795
        ZXO        0.000134
Length: 250000, dtype: float64